In [1]:
from elasticsearch import Elasticsearch
import pandas as pd
import numpy as np

es = Elasticsearch(hosts=['http://localhost:9200'], timeout=100)

/var/folders/vt/twcr0v015l9fsrh7yx2_tqw40000gn/T/ipykernel_4562/2181278079.py:5: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(hosts=['http://localhost:9200'], timeout=100)


In [2]:
df_ner = pd.read_csv('/dataset/attribute_text_filtering.csv')
df_ner

,product_name,ingredient,feature,skin_type,capacity,design,brand,expiry,origin,product_id,shop_id,type,skin_kind,is_5_star,num_sold_time,price
0,Sữa rửa mặt Simple lành tính sạch thoáng - cho...,x2 vitamin b5 vitamin e và pro amino acids,lành tính sạch thoáng cho da nhạy cảm,NaN,NaN,NaN,simple,03 năm,poland,8587034871,111138057,no_type,no_skin,0.908139,1.000000,99000.0
1,Sữa rửa mặt Simple giúp kiềm dầu và ngừa mụn h...,"chiết xuất cây hazel, kẽm, vitamin b3","kiềm dầu, ngừa mụn, cân bằng độ ẩm",da dầu dễ nổi mụn,NaN,NaN,simple,03 năm kể từ ngày sản xuất,poland,8287055694,111138057,no_type,no_skin,0.908295,0.552504,99000.0
2,Sữa rửa mặt nam 30Shine phân phối chính hãng S...,chiết xuất từ cây tràm trà chứa axit và glycol...,"làm sạch da hiệu quả, làm mờ vết thâm, giúp da...","cho da mụn, da dầu",80g,tuýp nhựa mềm có nắp đậy,skin dr tràm trà,2 năm kể từ ngày sản xuất,hàn quốc,9335942418,21080428,dạng kem,no_skin,0.945537,0.043619,244000.0
3,Sữa rửa mặt nam RHYS MAN Rhys Coconut Fresh hư...,"chiết xuất lô hội, tinh chất dầu dừa, tinh dầu...","giúp làm sạch da, giảm nhờn và giữ ẩm, giảm mụ...",NaN,10 ml,NaN,rhys man,2 năm kể từ ngày sản xuất,việt nam,19601743242,758111427,dạng sánh,Da hỗn hợp,0.933333,0.019386,119000.0
4,🌈🍒🍭 Sữa Rửa Mặt Senana Lấy Sạch Bụi Bẩn Giúp D...,NaN,"dưỡng da ẩm mượt tươi trẻ, cấp nước",NaN,60 g,tuýp nhỏ gọn,senana,NaN,nội địa đài loan,3330988133,107632788,no_type,no_skin,0.903042,0.141626,9900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2239,Sữa rửa mặt Kose Softymo Nhật Bản 220g,"white colagen và acid, các thành phần tự nhiên...","làm sạch sâu, làm trắng, làm mờ vết thâm đốm, ...",NaN,tuýp,"màu trắng, xanh và hồng",kose,NaN,nhật bản,5445658669,294293046,no_type,no_skin,0.977778,0.004696,82000.0
2240,Sữa rửa mặt La Roche-Posay Effaclar chuyên dàn...,"khoáng chất chống oxy hóa selen, glycerin","giúp ngăn ngừa và hỗ trợ điều trị mụn, làm sạc...",dành cho da dầu và da mụn,có nhiều dung tích để các bạn lựa chọn như 50m...,dạng gel tạo bọt,efaclar laroche posay,3 năm kể từ ngày sản xuất,pháp,10761076591,97333460,dạng gel,"Da mụn trứng cá, Da hỗn hợp, Dầu, Nhạy cảm",1.000000,0.000232,205000.0
2241,Sữa rửa mặt Kose Softymo Nhật Bản 220g,"tinh chất ngọc trai, hibiscus, li ti, silk ami...","tẩy sạch bụi bẩn, tăng cường độ ẩm, làm trắng ...",khô,20 g,"chai nhỏ, tiện dụng",kose,NaN,nhật bản,3455178555,315629668,no_type,no_skin,0.963303,0.004432,75000.0
2242,Sữa Rửa Mặt Làm Sáng Da Reihaku Hatomugi Facia...,hạt ý dĩ,"cung cấp độ ẩm và dưỡng chất cho da, giúp làm ...",NaN,NaN,NaN,reihaku hatomugi,3 năm từ nsx,nhật bản,6677227750,258099521,no_type,no_skin,0.974943,0.005385,79000.0


In [4]:
data = df_ner[df_ner['product_id']==3330988133]
for value in data.values:
    print(value)

['🌈🍒🍭 Sữa Rửa Mặt Senana Lấy Sạch Bụi Bẩn Giúp Da Trắng Sáng-Y3-K6T3' nan
 'dưỡng da ẩm mượt tươi trẻ, cấp nước' nan '60 g' 'tuýp nhỏ gọn' 'senana'
 nan 'nội địa đài loan' 3330988133 107632788 'no_type' 'no_skin'
 0.903041825095057 0.1416262789445342 9900.0]


In [5]:
list_used_col = ['product_name', 'ingredient', 'feature', 'skin_type', 'capacity', 'design',
                'brand', 'expiry', 'origin']


In [229]:
def recommend_product(product_id):
    must = []
    data = df_ner[df_ner['product_id']==product_id]
    for col in list_used_col:
        if pd.isnull(data[col].values[0]) == False:
            must.append({
                "match": {
                    col: {
                        'query': data[col].values[0]
                    }
                }
            })

    query_template = {
        'query': {
            'bool': {
                'should': must
            }
        },
        'size': 101  # Lấy 11 kết quả
    }
    result = es.search(index="ner_shopee", body=query_template)
    ranked_list = {}

    for i in range(1, 11):
        product_id = int(result['hits']['hits'][i]['_id'])
        ranked_list[product_id] = {'score': result['hits']['hits'][i]['_score'],
                                   'num_sold': result['hits']['hits'][i]['_source']['num_sold_time']}

        ranked_list[product_id]['score'] = ranked_list[product_id]['score']*0.7 + ranked_list[product_id]['num_sold']*0.3

    ranked_list = dict(sorted(ranked_list.items(), key=lambda x: x[1]['score'], reverse=True))

    return ranked_list

In [230]:
ranked_list = recommend_product(product_id=9644340745)

/var/folders/vt/twcr0v015l9fsrh7yx2_tqw40000gn/T/ipykernel_5692/1817017888.py:29: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="ner_shopee", body=query_template)


In [231]:
ranked_list

{3019169923: {'score': 13.919670552504039, 'num_sold': 0.0019709208400646},
 20739934444: {'score': 13.919172975767367, 'num_sold': 0.0011416262789445},
 10007498774: {'score': 12.770893660743134, 'num_sold': 0.0026494345718901},
 1542143520: {'score': 12.681190442649434, 'num_sold': 0.0129240710823909},
 2209295083: {'score': 12.674372995153474, 'num_sold': 0.0015616585891222},
 7121552405: {'score': 12.673713867528273, 'num_sold': 0.000463112547119},
 3790012157: {'score': 12.673636323101778, 'num_sold': 0.000333871836295},
 7772750102: {'score': 12.673536161550889, 'num_sold': 0.0001669359181475},
 7258612281: {'score': 12.360812253311792, 'num_sold': 0.0161550888529886},
 16870298456: {'score': 12.351261364781905, 'num_sold': 0.0002369413031771}}

In [233]:
def NDCG(y_true, y_pred, k=None):
    if y_true not in y_pred:
        return 0.0

    true_index = y_pred.index(y_true)
    ideal_sorted = sorted(y_pred, reverse=True)
    ideal_index = ideal_sorted.index(y_true)

    dcg = 1.0 / np.log2(true_index + 2)
    idcg = 1.0 / np.log2(ideal_index + 2)

    ndcg = dcg / idcg

    if k is not None:
        ideal_sorted = ideal_sorted[:k]
        y_pred = y_pred[:k]
        true_index = y_pred.index(y_true)
        ideal_index = ideal_sorted.index(y_true)
        dcg = 1.0 / np.log2(true_index + 2)
        idcg = 1.0 / np.log2(ideal_index + 2)
        ndcg = dcg / idcg

    return ndcg
def MRR(y_true, y_pred):
    if y_true not in y_pred:
        return 0

    rank = 0
    for index, value in enumerate(y_pred):
        if value == y_true:
            rank = 1 / (index + 1)
            break

    return rank

In [234]:
def PtopK(y_true, y_pred):
  if y_true in y_pred:
    return 1/len(y_pred)
  else:
    return 0
def RtopK(y_true, y_pred):
  if y_true in y_pred:
    return 1
  else:
    return 0

# Perform test

In [235]:
df_purchase = pd.read_csv('/dataset/data_purchase.csv')
df_purchase

,user_id,product_id,cmt_date
0,871725073.0,18424418643,2022-11-10 12:58:22
1,871725073.0,13006850994,2022-11-08 05:34:08
2,871725073.0,4571147728,2022-10-08 12:25:49
3,856269925.0,3330988133,2022-11-30 16:30:20
4,856269925.0,13613809516,2022-11-30 16:26:40
...,...,...,...
12758,41680.0,779015738,2020-07-14 09:10:27
12759,41680.0,845465208,2020-06-26 03:47:16
12760,40609.0,6312991956,2022-11-26 11:42:11
12761,40609.0,6934921300,2021-05-04 16:07:58


In [236]:
rtopk_val = []
rtopk_test = []
ptopk_val = []
ptopk_test = []
mrr_val = []
mrr_test = []
ndcg_val = []
ndcg_test = []

for user in df_purchase['user_id'].unique():
  temp = df_purchase[df_purchase['user_id']==user]
  x_val = temp.iloc[2, 1]
  y_val = temp.iloc[1, 1]
  y_val_pred = recommend_product(product_id=x_val.tolist())
  y_val_pred = list(y_val_pred.keys())[:10]
  rtopk_val.append(RtopK(y_val, y_val_pred))
  ptopk_val.append(PtopK(y_val, y_val_pred))
  mrr_val.append(MRR(y_val, y_val_pred))
  ndcg_val.append(NDCG(y_val, y_val_pred, 10))

  x_test = temp.iloc[1, 1]
  y_test = temp.iloc[0, 1]
  y_test_pred = recommend_product(product_id=x_test.tolist())
  y_test_pred = list(y_test_pred.keys())[:10]
  rtopk_test.append(RtopK(y_test, y_test_pred))
  ptopk_test.append(PtopK(y_test, y_test_pred))
  mrr_test.append(MRR(y_test, y_test_pred))
  ndcg_test.append(NDCG(y_test, y_test_pred, 10))


/var/folders/vt/twcr0v015l9fsrh7yx2_tqw40000gn/T/ipykernel_5692/1817017888.py:29: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="ner_shopee", body=query_template)


In [237]:
def calculate_mean(numbers):
    if not numbers:
        return 0.0
    return sum(numbers) / len(numbers)

print('rtopk val', calculate_mean(rtopk_val))
print('rtopk test', calculate_mean(rtopk_test))
print('ptopk val',calculate_mean(ptopk_val))
print('ptopk test', calculate_mean(ptopk_test))
print('mrr val', calculate_mean(mrr_val))
print('mrr test', calculate_mean(mrr_test))
print('ndcg val', calculate_mean(ndcg_val))
print('ndcg test', calculate_mean(ndcg_test))

rtopk val 0.09817813765182186
rtopk test 0.1305668016194332
ptopk val 0.009817813765182258
ptopk test 0.013056680161943437
mrr val 0.04803776267592058
mrr test 0.06397935945633311
ndcg val 0.17736968166132466
ndcg test 0.22842254764890105
